<a href="https://colab.research.google.com/github/andymaru/aluraflix/blob/main/telegram_chatbot_with_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de Bibliotecas


In [1]:
!pip install pytelegrambotapi
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00


# Baixando nosso dataset

In [2]:
!git clone https://github.com/gladson-sza/atrativos_turisticos_dataset.git

Cloning into 'atrativos_turisticos_dataset'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), 19.53 KiB | 3.25 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# Lendo nosso dataset

## Pandas


A biblioteca pandas é uma poderosa biblioteca em Python amplamente utilizada para análise de dados e manipulação de estruturas de dados tabulares, como DataFrames. Ela oferece funcionalidades para carregar, limpar, transformar, analisar e visualizar dados de maneira eficiente.

In [3]:
import pandas as pd

In [4]:
dataset = pd.read_csv('/content/atrativos_turisticos_dataset/dataset.csv', delimiter=';')

In [5]:
dataset.head()

,Pergunta,Resposta
0,Bom dia!,Bom dia! Como posso ajudar você hoje?
1,Boa tarde!,Boa tarde! Estou à disposição para responder à...
2,Boa noite!,Boa noite! Em que posso ser útil?
3,Oi!,Olá! Como posso ser útil a você?
4,"Olá, viajante!",Olá! Estou aqui para fornecer informações sobr...


# Bibliotecas para tratar strings



In [6]:
import string

# Bibliotecas de NLP

## NLTK
O Natural Language Toolkit (NLTK) é uma biblioteca em Python amplamente utilizada para processamento de linguagem natural (NLP). Ela oferece uma gama de ferramentas que tornam mais fácil a realização de análises textuais e tarefas de NLP, bem como a construção de sistemas de processamento de linguagem natural.

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Biblioteca para remoção de acentuação

In [8]:
from unidecode import unidecode

# Pré processamento de texto

In [9]:
def remove_pontuacao(text):
    texto_limpo = ''
    for palavra in text:
      if palavra not in string.punctuation:
        texto_limpo += palavra

    return texto_limpo

In [ ]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [10]:
def preprocessamento(texto):
    # Remove pontuações e símbolos
    texto = remove_pontuacao(texto)

    # Remover acentos
    texto = unidecode(texto)

    # Converte para minúsculo
    texto = texto.lower()

    # Tokenização
    tokens = word_tokenize(texto)

    # Remover stopwords
    stop_words = stopwords.words('portuguese')
    tokens = [token for token in tokens if token not in stop_words]

    # Juntar as palavras novamente em uma string
    texto_preprocessado = ' '.join(tokens)

    return texto_preprocessado

In [11]:
dataset["Pergunta_Preprocessada"] = dataset["Pergunta"].apply(preprocessamento)

In [12]:
dataset.head()

,Pergunta,Resposta,Pergunta_Preprocessada
0,Bom dia!,Bom dia! Como posso ajudar você hoje?,bom dia
1,Boa tarde!,Boa tarde! Estou à disposição para responder à...,boa tarde
2,Boa noite!,Boa noite! Em que posso ser útil?,boa noite
3,Oi!,Olá! Como posso ser útil a você?,oi
4,"Olá, viajante!",Olá! Estou aqui para fornecer informações sobr...,ola viajante


## Vetorização

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dataset["Pergunta_Preprocessada"])

## Método para obter uma resposta a partir de uma pergunta

In [15]:
def obter_resposta(pergunta):
    # Pré processa a sentença da pergunta
    pergunta_processada = preprocessamento(pergunta)

    # Transforma a pergunta para um vetor
    pergunta_vector = vectorizer.transform([pergunta_processada])

    # Calcula a similaridade do cosseno
    similaridades = cosine_similarity(pergunta_vector, tfidf_matrix)

    # Obtém o índice da pergunta mais similar
    pergunta_index = similaridades.argmax()

    # Devolve a resposta para o usuário
    return dataset["Resposta"].iloc[pergunta_index]

## Testando Chatbot

In [16]:
flags = ['Fechar', 'Sair', 'Tchau']
end = False

while not end:
    question = input()
    if question in flags:
        end = True
        print('Finalizando Chat')
        continue

    answer = obter_resposta(question)
    print(answer)

oi
Olá! Como posso ser útil a você?
ohayo
Bom dia! Como posso ajudar você hoje?
sumimassen
Bom dia! Como posso ajudar você hoje?
konnichiwa
Bom dia! Como posso ajudar você hoje?
good afternoon
Bom dia! Como posso ajudar você hoje?
boa tarde
Boa tarde! Estou à disposição para responder às suas perguntas.
boa noite
Boa noite! Em que posso ser útil?
Olar
Bom dia! Como posso ajudar você hoje?
fala seu bot com ia
Falo vários idiomas, incluindo português e inglês, para ajudar viajantes de diferentes origens.
falar em ingles
Bom dia! Como posso ajudar você hoje?
good morning
Bom dia! Como posso ajudar você hoje?
bom dia pistoleiro
Bom dia! Como posso ajudar você hoje?
oiiiiiiiiiiiiiiiiiiiiii
Bom dia! Como posso ajudar você hoje?
Boa noite pistoleiro
Boa noite! Em que posso ser útil?
fechar
Bom dia! Como posso ajudar você hoje?
Fechar
Finalizando Chat


# Integrando com Telegram

In [17]:
import telebot

API_KEY = '6759433623:AAE5_qcZiNXl03nFwj1CjCwTMQMZxj355Uc'
bot = telebot.TeleBot(API_KEY)

In [18]:
@bot.message_handler(func=lambda message: True)
def default(message):
    resposta = obter_resposta(message.text)

    bot.reply_to(
        message,
        resposta
    )

In [19]:
bot.polling()